In [58]:
import csv
import pandas as pd
import random
import pickle
import numpy as np
import tensorflow as tf
import zipfile
import warnings
warnings.simplefilter("ignore")


from tensorflow.keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [59]:
EMBEDDING_DIM = 100
MAXLEN = 16
TRUNCATING = 'post'
PADDING = 'post'
OOV_TOKEN = "<OOV>"
MAX_EXAMPLES = 5000
TRAINING_SPLIT = 0.9

In [60]:
!wget https://github.com/aldofdp07/Sentiment-Analysis-Part-2/blob/main/tugas-analisis-sentimen-text-mining-kelas-b.zip

--2022-11-12 14:48:59--  https://github.com/aldofdp07/Sentiment-Analysis-Part-2/blob/main/tugas-analisis-sentimen-text-mining-kelas-b.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-11-12 14:48:59 ERROR 404: Not Found.



In [61]:
!unzip /content/tugas-analisis-sentimen-text-mining-kelas-b.zip

unzip:  cannot find or open /content/tugas-analisis-sentimen-text-mining-kelas-b.zip, /content/tugas-analisis-sentimen-text-mining-kelas-b.zip.zip or /content/tugas-analisis-sentimen-text-mining-kelas-b.zip.ZIP.


In [112]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
train.dropna(axis=1)
test.dropna(axis=1)
train.head()

,Id,Sentiment,pasangan_calon,text_tweet,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,1,negative,Agus-Sylvi,'Banyak akun kloning seolah2 pendukung #agussi...,NaN,NaN,NaN,NaN,NaN,NaN
1,2,negative,Agus-Sylvi,'#agussilvy bicara apa kasihan yaa...lap itu a...,NaN,NaN,NaN,NaN,NaN,NaN
2,3,negative,Agus-Sylvi,'Kalau aku sih gak nunggu hasil akhir QC tp la...,NaN,NaN,NaN,NaN,NaN,NaN
3,4,negative,Agus-Sylvi,'Kasian oh kasian dengan peluru 1milyar untuk ...,NaN,NaN,NaN,NaN,NaN,NaN
4,5,negative,Agus-Sylvi,'Maaf ya pendukung #AgusSilvy..hayo dukung #An...,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
print(train["text_tweet"])

0      'Banyak akun kloning seolah2 pendukung #agussi...
1      '#agussilvy bicara apa kasihan yaa...lap itu a...
2      'Kalau aku sih gak nunggu hasil akhir QC tp la...
3      'Kasian oh kasian dengan peluru 1milyar untuk ...
4      'Maaf ya pendukung #AgusSilvy..hayo dukung #An...
                             ...                        
625    'Setelah KO di beberapa Polling kini mereka go...
626    'program asal ngomong begini jadinya pemotonga...
627    'Waaaah ternyata dia sudah sadar kalo selama i...
628    'Belajar mulai dari yang sederhana: Kursi dipa...
629    'Lho Lagu Anies-Sandi Menjiplak Nyanyian Yahud...
Name: text_tweet, Length: 630, dtype: object


In [87]:
def train_val_split(sentences, labels, training_split):
    
    train_data, validation_data, train_labels, validation_labels = train_test_split(sentences, labels, train_size=training_split, random_state=24)
    return train_data, validation_data, train_labels, validation_labels
    
    return train_data, validation_data, train_labels, validation_labels

In [88]:
# Test your function
sentences = train["text_tweet"]
labels = train["Sentiment"]
train_sentences, val_sentences, train_labels, val_labels = train_val_split(sentences, labels, TRAINING_SPLIT)

print(f"There are {len(train_sentences)} sentences for training.\n")
print(f"There are {len(train_labels)} labels for training.\n")
print(f"There are {len(val_sentences)} sentences for validation.\n")
print(f"There are {len(val_labels)} labels for validation.")

There are 567 sentences for training.

There are 567 labels for training.

There are 63 sentences for validation.

There are 63 labels for validation.


In [89]:
def fit_tokenizer(train_sentences, oov_token):
    
    ### START CODE HERE
        # Instantiate the Tokenizer class, passing in the correct values for num_words and oov_token
    tokenizer = Tokenizer(oov_token = oov_token, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
        # Fit the tokenizer to the training sentences
    tokenizer.fit_on_texts(train_sentences)
    ### END CODE HERE  
    return tokenizer

In [90]:
# Test your function
tokenizer = fit_tokenizer(train_sentences, OOV_TOKEN)

word_index = tokenizer.word_index
vocab_size = len(word_index)

print(f"Vocabulary contains {vocab_size} words\n")
print("<OOV> token included in vocabulary" if "<OOV>" in word_index else "<OOV> token NOT included in vocabulary")

Vocabulary contains 3092 words

<OOV> token included in vocabulary


In [91]:
print(word_index)

{'<OOV>': 1, "'": 2, 'ahy': 3, 'yang': 4, 'ahok': 5, 'ahokdjarot': 6, 'face': 7, 'dan': 8, 'di': 9, 'with': 10, 'tidak': 11, 'untuk': 12, 'pilkada': 13, "ahokdjarot'": 14, 'ini': 15, 'jakarta': 16, 'djarot': 17, 'https': 18, 'pilkadadki2017': 19, 'pak': 20, 'dki': 21, "pilkadadki2017'": 22, 'hand': 23, 'smiling': 24, 'victory': 25, 'kalah': 26, 'com': 27, 'jadi': 28, 'anies': 29, 'itu': 30, 'dari': 31, 'dengan': 32, "ahy'": 33, 'indonesia': 34, 'pilkadadki': 35, 'www': 36, 'sudah': 37, 'http': 38, 'pada': 39, 'menang': 40, 'ada': 41, 'basuki': 42, 'bunga': 43, 'eyes': 44, 'dalam': 45, 'ya': 46, 'masih': 47, 'bisa': 48, 'pilih': 49, 'instagram': 50, 'btp': 51, 'pendukung': 52, 'ke': 53, 'p': 54, 'saya': 55, 'karena': 56, 'pemimpin': 57, 'tapi': 58, 'mau': 59, 'bukan': 60, 'mereka': 61, 'orang': 62, 'gubernur': 63, 'lebih': 64, 'jangan': 65, 'sby': 66, 'baru': 67, 'lagi': 68, 'apa': 69, 'hands': 70, 'warga': 71, 'stuck': 72, 'out': 73, 'sandi': 74, 'badja': 75, 'sama': 76, 'no': 77, 'tet

In [92]:
def seq_pad_and_trunc(sentences, tokenizer, padding, truncating, maxlen):
    
    ### START CODE HERE     
    # Convert sentences to sequences
    sequences = tokenizer.texts_to_sequences(sentences)
    # Pad the sequences using the correct padding, truncating and maxlen
    pad_trunc_sequences = pad_sequences(sequences, padding=padding, truncating=truncating, maxlen=maxlen)
    ### END CODE HERE
    
    return pad_trunc_sequences

In [93]:
# Test your function
train_pad_trunc_seq = seq_pad_and_trunc(train_sentences, tokenizer, PADDING, TRUNCATING, MAXLEN)
val_pad_trunc_seq = seq_pad_and_trunc(val_sentences, tokenizer, PADDING, TRUNCATING, MAXLEN)

print(f"Padded and truncated training sequences have shape: {train_pad_trunc_seq.shape}\n")
print(f"Padded and truncated validation sequences have shape: {val_pad_trunc_seq.shape}")

Padded and truncated training sequences have shape: (567, 16)

Padded and truncated validation sequences have shape: (63, 16)


In [94]:
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [95]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
val_labels = le.fit_transform(val_labels)

In [109]:
# GRADED FUNCTION: create_model
def create_model(vocab_size, embedding_dim, maxlen):
    
    ### START CODE HERE
    model = tf.keras.Sequential([ 
        # This is how you need to set the Embedding layer when using pre-trained embeddings
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen, trainable=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

   # model = Sequential()
    #model.add(Embedding(vocab_size+1, embedding_dim, input_length=maxlen, weights=[embeddings_matrix], trainable=True))
    #model.add(SpatialDropout1D(0.5))
    #model.add(LSTM(100, dropout=0.5, recurrent_dropout=0.5))
    #model.add(Dense(32, activation='relu'))
    #model.add(Dense(6, activation='softmax'))
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer='adam',
                  metrics=['accuracy']) 

    ### END CODE HERE
    return model

In [110]:
# Create your untrained model
model = create_model(vocab_size, EMBEDDING_DIM, MAXLEN)

# Train the model and save the training history
history = model.fit(train_pad_trunc_seq, train_labels, epochs=15, validation_data=(val_pad_trunc_seq, val_labels))

Epoch 1/15
18/18 [==============================] - 4s 74ms/step - loss: 0.6897 - accuracy: 0.5344 - val_loss: 0.6726 - val_accuracy: 0.5873
Epoch 2/15
18/18 [==============================] - 1s 37ms/step - loss: 0.5031 - accuracy: 0.7478 - val_loss: 0.6474 - val_accuracy: 0.6825
Epoch 3/15
18/18 [==============================] - 1s 38ms/step - loss: 0.1333 - accuracy: 0.9418 - val_loss: 0.5855 - val_accuracy: 0.7619
Epoch 4/15
18/18 [==============================] - 1s 38ms/step - loss: 0.0250 - accuracy: 0.9982 - val_loss: 1.2386 - val_accuracy: 0.7143
Epoch 5/15
18/18 [==============================] - 1s 37ms/step - loss: 0.0060 - accuracy: 0.9982 - val_loss: 1.7800 - val_accuracy: 0.7460
Epoch 6/15
18/18 [==============================] - 1s 39ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 1.6775 - val_accuracy: 0.7143
Epoch 7/15
18/18 [==============================] - 1s 38ms/step - loss: 3.7922e-05 - accuracy: 1.0000 - val_loss: 1.6952 - val_accuracy: 0.7143
Epoch 8/1

In [113]:
test.head()

,Id,pasangan_calon,text_tweet,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,631,Anies-Sandi,'Didiklah orang muda dengan akhlak dan integri...,NaN,NaN,NaN,NaN
1,632,Anies-Sandi,'#PilkadaDKI2017 Anies Dipolisikan karena Sebu...,NaN,NaN,NaN,NaN
2,633,Anies-Sandi,'Siapa menebar FITNAH dia akan menuai KALAH #p...,NaN,NaN,NaN,NaN
3,634,Anies-Sandi,'Pemimpin yang Tegas ataukah Pemimpi yang Tega...,NaN,NaN,NaN,NaN
4,635,Anies-Sandi,'Lah Ini Malah Ada Pelatihan OK-OCE Sabun Al-M...,Ini Malah Ada Pelatihan OK-OCE Sabun Al-Maida...,NaN,NaN,NaN


In [114]:
sent = test["text_tweet"]

tokenizer = fit_tokenizer(sent, OOV_TOKEN)

test_pad_trunc_seq = seq_pad_and_trunc(sent, tokenizer, PADDING, TRUNCATING, MAXLEN)


In [115]:
x = np.argmax(model.predict(test_pad_trunc_seq), axis=-1)

9/9 [==============================] - 1s 11ms/step


In [116]:
test['Sentiment']=x

In [117]:
test = test.drop(['pasangan_calon','text_tweet'],axis = 1)

In [118]:
test.head()

,Id,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Sentiment
0,631,NaN,NaN,NaN,NaN,1
1,632,NaN,NaN,NaN,NaN,1
2,633,NaN,NaN,NaN,NaN,0
3,634,NaN,NaN,NaN,NaN,1
4,635,Ini Malah Ada Pelatihan OK-OCE Sabun Al-Maida...,NaN,NaN,NaN,0


In [119]:
test.to_csv('submission_Aldofdp.csv')